In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

--2024-11-27 01:16:56--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.122.207, 172.253.63.207, 142.250.31.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.122.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   249MB/s    in 0.3s    

2024-11-27 01:16:56 (249 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
from PIL import Image
import numpy as np

x_train = []
y_train = []

cats_dir = os.path.join(train_dir + "/cats")
dogs_dir = os.path.join(train_dir + "/dogs")
i = 0
while i < min(len(os.listdir(cats_dir)), len(os.listdir(dogs_dir))):
  if i % 2 == 0:
    im = Image.open(os.path.join(cats_dir, os.listdir(cats_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))

    x_train.append(np.array(im_resized))
    y_train.append(1)
  else:
    im = Image.open(os.path.join(dogs_dir, os.listdir(dogs_dir)[i])).convert("RGB")
    im_resized = im.resize((150, 150))

    x_train.append(np.array(im_resized))
    y_train.append(0)
  i += 1

In [4]:
x_train = np.array(x_train)
y_train = np.array(y_train)


In [5]:
split_index = int(0.8 * len(x_train))
x_val = x_train[split_index:]
y_val = y_train[split_index:]
x_train = x_train[:split_index]
y_train = y_train[:split_index]

In [6]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
model = tf.keras.Sequential([
    pretrained_model,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [8]:
pretrained_model.trainable = False

In [9]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [10]:
history = model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    epochs = 10,
    batch_size = 32
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 98s 4s/step - accuracy: 0.8044 - loss: 0.4211 - val_accuracy: 0.9350 - val_loss: 0.1650
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.9606 - loss: 0.0832 - val_accuracy: 0.9550 - val_loss: 0.1185
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 140s 4s/step - accuracy: 0.9867 - loss: 0.0287 - val_accuracy: 0.9550 - val_loss: 0.1459
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 1.0000 - loss: 0.0125 - val_accuracy: 0.9550 - val_loss: 0.1406
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 142s 4s/step - accuracy: 1.0000 - loss: 0.0060 - val_accuracy: 0.9550 - val_loss: 0.1447
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 144s 4s/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 0.9350 - val_loss: 0.1624
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 140s 4s/step - accuracy: 1.0000 - loss: 0.0027 - val_accuracy: 0.9500 - val_loss: 0.1592
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.9600 - val

In [13]:
pretrained_model.trainable = True

In [14]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001),
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [15]:
history_fine_tune = model.fit(
    x_train, y_train,
    validation_data = (x_val, y_val),
    epochs = 5,
    batch_size = 32
)

Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 360s 13s/step - accuracy: 0.9624 - loss: 0.1097 - val_accuracy: 0.9450 - val_loss: 0.1788
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 316s 13s/step - accuracy: 0.9961 - loss: 0.0249 - val_accuracy: 0.9350 - val_loss: 0.1985
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 321s 13s/step - accuracy: 1.0000 - loss: 0.0101 - val_accuracy: 0.9450 - val_loss: 0.2041
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 317s 12s/step - accuracy: 0.9997 - loss: 0.0084 - val_accuracy: 0.9400 - val_loss: 0.2036
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 325s 13s/step - accuracy: 1.0000 - loss: 0.0050 - val_accuracy: 0.9350 - val_loss: 0.2052


In [16]:
sample_image = x_val[0:1]
prediction = model.predict(sample_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [17]:
print("Prediction(0 = Cat, 1 = Dog):", "Dog" if prediction[0] > 0.5 else "Cat")

Prediction(0 = Cat, 1 = Dog): Dog
